In [28]:
# Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, EfficientNetB0, VGG16
from tensorflow.keras.models import Model
import os
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from itertools import cycle
from sklearn.metrics import roc_curve, auc, roc_auc_score, RocCurveDisplay
from sklearn.preprocessing import label_binarize


import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from keras_tuner import HyperParameters 


# ----------- CONSTANTS ----------------

# define directory structure
TRAIN_DIR = "PROCESSED_DATA/TRAINING_DATA/TRAINING_AUGMENTED_DATA"
VALID_DIR = "PROCESSED_DATA/VALIDATION_DATA/"
TEST_DIR = "PROCESSED_DATA/TEST_DATA/"

# Image Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NORMALIZE_FLAG = True
NO_FRILLS_DATAGEN = ImageDataGenerator()
NORM_DATAGEN = ImageDataGenerator(rescale=1./255)

TRAIN_GENERATOR = load_data(TRAIN_DIR)
VAL_GENERATOR = load_data(VALID_DIR)
TEST_GENERATOR = load_data(TEST_DIR,shuffle_flag=False)
hp = HyperParameters()


def load_data(directory,shuffle_flag=True):
    '''
    Param: 
        - directory - str, 
        - shuffle_flag - boolean, introduces constrolled stochasticity
    '''
    if NORMALIZE_FLAG == True:
        generator = NORM_DATAGEN.flow_from_directory(
        directory,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',shuffle=shuffle_flag)
        return generator
    else:
        generator = NO_FRILLS_DATAGEN.flow_from_directory(
        directory,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',shuffle=shuffle_flag)
        return generator

    
# MODEL BUILDING Functions -------------------

def build_tunable_cnn(hp):
    '''
    Params: hp - keras HyperParameter object
    Use: Builds cnn with tunable hyperparameter for use with bayesian optimization
    '''
    model = Sequential([
        Input(shape=(224, 224, 3)),
        Conv2D(32, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(hp.Float("dropout", min_value=0.2, max_value=0.5, step=0.1)), #tune dropout
        Dense(TRAIN_GENERATOR.num_classes, activation='softmax')
    ])
    
    # tune learning rate, batch size
    learning_rate = hp.Choice('lr', values=[1e-2, 1e-3, 1e-4])
    batch_size = hp.Choice('batch_size', values=[16, 32])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model
    
def build_best_model():
    '''Builds cnn model from best performing hyperparameters'''
    
    # us keras bayesian tuner
    tuner = kt.BayesianOptimization(
        build_tunable_cnn,
        objective='val_accuracy',  # tune by improving validation accuracy
        max_trials=20,  # num different hp combos to try
        executions_per_trial=1,  # run each model once
        directory='bayesian_tuning',
        project_name='lr_and_drop_tuning'
    )
    
    # search hp combos
    tuner.search(TRAIN_GENERATOR, validation_data=VAL_GENERATOR, epochs=10)
    
    # get best hps
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    # save them
    best_hps_dict = {'best_lr': best_hps.get('lr'),
                    'best_dropout': best_hps.get('dropout'),
                    'best_batch_size': best_hps.get('batch_size')}
    
    # make final model with the best drop out, learning rate and batch size
    best_model = tuner.hypermodel.build(best_hps)
    best_model_training_history = best_model.fit(TRAIN_GENERATOR, validation_data=VAL_GENERATOR, epochs=10, batch_size=best_hps.get('batch_size'))

    return best_hps_dict, best_model, best_model_training_history


def evaluate_model(model, filename = "best_model.h5"):
    '''
    Saves model to h5 file, returns test accuracy loss and test accuracy
    '''
    # evaluate on test data
    test_loss, test_acc = model.evaluate(TEST_GENERATOR)
    print(f"Test Accuracy: {test_acc:.4f}")
    
    # save to file
    model.save(filename)

    return test_loss, test_acc


def main():

    # best model
    best_hps_dict, best_model, best_model_training_history = build_best_model()

    print(f'best parameters:\n {best_hps_dict}')
    
    test_loss, test_acc = evaluate_model(best_model)
    print(f'test_loss: {test_loss}, test_acc: {test_acc}')
    
    best_model.save("simple_cnn_best_model_bayes_optimization.h5")
    
if __name__ == "__main__":
    main()


Trial 20 Complete [00h 11m 25s]
val_accuracy: 0.4545454680919647

Best val_accuracy So Far: 0.4840908944606781
Total elapsed time: 03h 55m 35s
Epoch 1/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 68s 323ms/step - accuracy: 0.1330 - loss: 2.3624 - val_accuracy: 0.2705 - val_loss: 2.1991
Epoch 2/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 71s 341ms/step - accuracy: 0.2729 - loss: 2.1061 - val_accuracy: 0.3318 - val_loss: 2.0641
Epoch 3/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 72s 345ms/step - accuracy: 0.3395 - loss: 1.9457 - val_accuracy: 0.3886 - val_loss: 1.9840
Epoch 4/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 68s 326ms/step - accuracy: 0.4189 - loss: 1.7702 - val_accuracy: 0.4000 - val_loss: 1.9555
Epoch 5/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 67s 323ms/step - accuracy: 0.4485 - loss: 1.6328 - val_accuracy: 0.4114 - val_loss: 1.9632
Epoch 6/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 67s 323ms/step - accuracy: 0.5177 - loss: 1.4669 - val_accuracy: 0.4114 - val_loss: 1.9182
Epoch 7/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 67s 324ms/step - accuracy: 0.57

/Users/keeganveazey/Desktop/ds_capstone/BugBot/bugbot_env/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.3424 - loss: 2.2733 


Test Accuracy: 0.3682


test_loss: 2.2131574153900146, test_acc: 0.36818182468414307
